# langfree

> Utilities to help you work with your OpenAI language model data outside LangSmith.

This library is specifically tested against OpenAI and LangSmith.  Many utitlies are centered around extracting data from the terminal [ChatOpenAI](https://api.python.langchain.com/en/latest/chat_models/langchain.chat_models.openai.ChatOpenAI.html) runs in LangChains.  This library is experimental and may change.

## Install

```sh
pip install langfree
```

## How to use

Coming soon

## Contributing

This library was created with [nbdev](https://nbdev.fast.ai/).  PRs will need to add appropriate docs, tests and code in notebooks using nbdev.  If you are not familiar with nbdev, please consider opening an issue.  See [Contributing.md](https://github.com/parlance-labs/langfree/blob/main/CONTRIBUTING.md) for further guidelines.